# **Classification of stroke diseases**

> **In the first stage, we conducted a complete and detailed exploration and analysis of stroke disease data, and the results we obtained are very useful information. You can view this kernel through the following link:
https://www.kaggle.com/alimohammedbakhiet/eda-for-stroke-dataset**

> **In the second stage, we will apply machine learning algorithms and neural networks to craft that data and we will work hard to get the highest results in accuracy for the test data.**

> **Let's have fun...**

> **In the first step we will do the usual things like reading the data and cleaning the data, and then we will move on to splitting the data and creating the models:**

In [ ]:
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [ ]:
data=pd.read_csv("../input/stroke-prediction-dataset/healthcare-dataset-stroke-data.csv")

In [ ]:
data.head(5)

**We will now drop some columns and replace some of them and transform the data.**

In [ ]:
data=data.drop(["id"], axis=1)

In [ ]:
# Here we will replace one column with another and drop the original.
bmi_median = data.bmi.median()
data['bmi_median'] = data.bmi.fillna(bmi_median)

In [ ]:
data.isnull().sum()

In [ ]:
data=data.drop(["bmi"], axis=1)

In [ ]:
data.isnull().sum()

# **Here I convert the data.**

In [ ]:
data["gender"]=data["gender"].map({"Male":0 , "Female":1 , "Other":2})
data["ever_married"]=data["ever_married"].map({"Yes":1 , "No":0 })
data["work_type"]=data["work_type"].map({'Private':0, 'Self-employed':1, 'Govt_job':2, 'children':3, 'Never_worked':4 })
data["smoking_status"]=data["smoking_status"].map({'formerly smoked':0, 'never smoked':1, 'smokes':2, 'Unknown':3 })
data["Residence_type"]=data["Residence_type"].map({'Urban':0, 'Rural':1})

In [ ]:
data.head()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix

In [ ]:
target=data["stroke"]
features=data.drop(["stroke"],axis=1)

In [ ]:
scaler = StandardScaler(copy=True, with_mean=True, with_std=True)
X = scaler.fit_transform(features)

# **Let's start building some machine learning algorithms:**

# **1 . Random Forest Algorithm**

In [ ]:
x_train,x_test, y_train, y_test = train_test_split(X,target,test_size=0.25,random_state=0)

In [ ]:
print(x_train.shape,x_test.shape,y_train.shape,y_test.shape)

In [ ]:
x_val, x_test, y_val, y_test=train_test_split(x_test, y_test, test_size=0.25)

In [ ]:
print( x_val.shape, x_test.shape, y_val.shape, y_test.shape)

**Since we have some outliers in our data and I noted them in the first stage, I'm going to use Random Forest because they are good at dealing with outliers.**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [ ]:
RandomForestClassifierModel=RandomForestClassifier()

In [ ]:
parameters = {
    "n_estimators":[50,70,100,150,200],
    "max_depth":[7,11,13,15,32,None]
    
}

In [ ]:
#  I will pass the classifier and parameters and the number of iteration in the GridSearchCV method.
cv = GridSearchCV(RandomForestClassifierModel,parameters,cv=5)
cv.fit(x_train, y_train)

In [ ]:
#I have defined the method for printing all the iteration done and scores in each iteration.
def display(results):
    print(f'Best parameters are: {results.best_params_}')
    print("\n")
    mean_score = results.cv_results_['mean_test_score']
    std_score = results.cv_results_['std_test_score']
    params = results.cv_results_['params']
    for mean,std,params in zip(mean_score,std_score,params):
        print(f'{round(mean,3)} + or -{round(std,3)} for the {params}')

In [ ]:
display(cv)

In [ ]:
RandomForestClassifierModel=RandomForestClassifier(n_estimators=70, criterion='gini', max_depth=7,
                                min_samples_split=2, min_samples_leaf=1,min_weight_fraction_leaf=0.0,
                                max_features='auto',max_leaf_nodes=7,min_impurity_decrease=0.0,
                                min_impurity_split=None, bootstrap=True,oob_score=False, n_jobs=-1,
                                random_state=0, verbose=0,warm_start=True)
RandomForestClassifierModel.fit(x_train, y_train)

In [ ]:
#Calculating Details
print('RandomForestClassifierModel Train Score is : ' , RandomForestClassifierModel.score(x_train, y_train))
print('RandomForestClassifierModel Test Score is : ' , RandomForestClassifierModel.score(x_test, y_test))
# This instruction calculates the percentage of importance for each of the features.
print('RandomForestClassifierModel features importances are : ' , RandomForestClassifierModel.feature_importances_)

**The accuracy of the test data reached 94%.**

In [ ]:
#Calculating Prediction
y_pred = RandomForestClassifierModel.predict(x_test)
y_pred_prob = RandomForestClassifierModel.predict_proba(x_test)
print('Predicted Value for RandomForestClassifierModel is : ' , y_pred[:10])
print("real values of y_test>>>>>>>>>>>>>>>>>>>>>>>>>>>is : \n" ,y_test[:10] )
print('Prediction Probabilities Value for RandomForestClassifierModel is : ' , y_pred_prob[:10])

In [ ]:
#Calculating Confusion Matrix
from sklearn.metrics import confusion_matrix,classification_report,plot_confusion_matrix
confusion_matrix(y_test,y_pred)

In [ ]:
plot_confusion_matrix(RandomForestClassifierModel,x_test,y_test);

# **Let's build a deep learning algorithm**

In [ ]:
# Now we are going to use a neural network for classification
# Here we will call the libraries that we need.
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from keras.models import Sequential # empty neural network
from keras.layers import Dense # layer constitution
import keras 
from keras.layers import Dropout
from keras import regularizers

In [ ]:
x_train = keras.utils.normalize(x_train, axis=1)

In [ ]:
model= Sequential([
    Dense(1000, activation='relu', input_shape=(10,)),
    Dropout(0.5),
    Dense(100, activation='relu'),
    Dropout(0.5),
    Dense(1000, activation='relu'),
    Dropout(0.3),
    Dense(7, activation='relu'),
    Dense(1, activation='sigmoid'),
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
              
hist = model.fit(x_train, y_train,
          batch_size=10, epochs=40,
          validation_data=(x_val, y_val))

In [ ]:
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper right')
plt.show()

In [ ]:
plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='lower right')
plt.show()

**It was a fast neural network to extract quick but good results.**

# **The end**

**I applied some quick algorithms and extracted an accuracy rate of 94% of the evaluation data.
thank you for your time**